In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import numpy as np
import scipy as sp
import scipy.stats
import matplotlib.pyplot as plt

import holodeck as holo
from holodeck import plot, host_relations
from holodeck.discrete import population
from holodeck.constants import MSOL

log = holo.log
log.setLevel(log.WARNING)

## Plot Scaling Relation

In [ ]:
mmbulge = host_relations.MMBulge_Standard()

mbulge = np.logspace(8, 13, 100) * MSOL

class host:
    pass

host.mbulge = mbulge
mbh = mmbulge.mbh_from_host(host, True)

fig, ax = plot.figax()
ax.plot(mbulge/MSOL, mbh/MSOL)

plt.show()

In [ ]:
relation = host_relations.MMBulge_Redshift_MM2013(zplaw=1.0)
NUM = 1e3

class host:
    mbulge = np.logspace(8, 13, int(NUM)) * MSOL
    redz = 10.0 ** np.random.uniform(-2, 1, mbulge.size)

mbh = relation.mbh_from_host(host, scatter=True)

smap = plot.smap(host.redz, cmap='bwr', log=True)
colors = smap.to_rgba(host.redz)

fig, ax = plot.figax()
ax.scatter(host.mbulge/MSOL, mbh/MSOL, color=colors)

plt.colorbar(smap, ax=ax)

plt.show()

## Discrete / Illustris Populations

### Reset Masses of Illustris-Based Binary Population

In [ ]:
pop = population.Pop_Illustris()
ill_name = os.path.basename(pop._fname).split('_')[1]
print("Loaded", pop.size, "binaries from Illustris", ill_name)

mod_resamp = population.PM_Resample(resample=2.0)
pop.modify(mod_resamp)
print("Population now has", pop.size, "elements")

In [ ]:
mmbulge = host_relations.MMBulge_MM2013()
mod_MM2013 = population.PM_Mass_Reset(mmbulge, scatter=True)
pop.modify(mod_MM2013)
# plot.plot_mbh_scaling_relations(pop)
plt.show()

### Redshift dependent mass evolution

In [ ]:
mbulge = pop.mbulge
redz = pop.redz[:, np.newaxis] * np.ones_like(mbulge)
mbef = pop.mass.copy()

mmbulge = host_relations.MMBulge_Redshift_MM2013(zplaw=2.0)
mod_MM2013 = population.PM_Mass_Reset(mmbulge, scatter=True)
pop.modify(mod_MM2013)
maft = pop.mass.copy()


mbulge, mbef, maft, redz = [vv.flatten() for vv in [mbulge, mbef, maft, redz]]
cmap = plot.smap(redz, cmap='bwr', log=True)
colors = cmap.to_rgba(redz)

fig, axes = plot.figax(figsize=[8, 6], nrows=2)

for ax, mval in zip(axes, [mbef, maft]):
    ax.scatter(mbulge/MSOL, mval/MSOL, color=colors, alpha=0.1, s=10)

plt.show()

## Bulge Fractions

In [ ]:
bulge_frac = host_relations.BF_Sigmoid(0.3, 1.0, width_dex=1.0)
mmbulge = host_relations.MMBulge_KH2013(bulge_frac=bulge_frac)

In [ ]:
NUM = 1e5
SCATTER = True
bulge_frac_const = host_relations.BF_Constant(0.5)
mmbulge_const = host_relations.MMBulge_KH2013(bulge_frac=bulge_frac_const)

bulge_frac_sigma = host_relations.BF_Sigmoid(0.3, 1.0)
mmbulge_sigma = host_relations.MMBulge_KH2013(bulge_frac=bulge_frac_sigma)

extr = [8.0, 12.0]
# mstar = np.random.uniform(8, 12, size=int(NUM))
mstar = np.random.uniform(0.0, 1.0, size=int(NUM))
mstar = np.power(mstar, 5.0)
mstar = np.diff(extr) * mstar + extr[0]
mstar = 10.0 ** mstar
mbhs_const = mmbulge_const.mbh_from_mstar(mstar*MSOL, scatter=SCATTER)/MSOL
mbhs_sigma = mmbulge_sigma.mbh_from_mstar(mstar*MSOL, scatter=SCATTER)/MSOL

fig, ax = plot.figax(xlabel='Stellar Mass [$M_\odot$]', ylabel='Black-Hole Mass [$M_\odot$]')
# ax.scatter(mstar, mbhs_const, s=2, alpha=0.5)
# ax.scatter(mstar, mbhs_sigma, s=2, alpha=0.5)

bins = np.logspace(*extr, 31)
hist, *_ = np.histogram(mbhs_const, bins=bins)
print(hist.shape)

qq = sp.stats.norm.cdf([-2, -1, 0, +1, +2])
bin_cents = holo.utils.midpoints(bins, log=True)
labels = [
    f'constant [{bulge_frac_const._bulge_mass_frac}]',
    f'sigmoid [{bulge_frac_sigma._bulge_frac_lo:.2f}, {bulge_frac_sigma._bulge_frac_hi:.2f}]'
]
for jj, (mbhs, lab) in enumerate(zip([mbhs_const, mbhs_sigma], labels)):
    conf = np.zeros((bins.size-1, 5))
    for ii in range(bins.size-1):
        lo = bins[ii]
        hi = bins[ii+1]
        sel = (lo < mstar) & (mstar < hi)
        conf[ii, :] = np.percentile(mbhs[sel], 100 * qq)

    cc, = ax.plot(bin_cents, conf[:, 2], label=lab)
    cc = cc.get_color()
    ax.fill_between(bin_cents, conf[:, 1], conf[:, -2], color=cc, alpha=0.25, lw=2.0)
    ax.fill_between(bin_cents, conf[:, 0], conf[:, -1], color=cc, alpha=0.25, lw=2.0)

ax.legend(title='bulge fraction')
plt.show()